In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"
# model_dir = "../../../Model/THUDM/glm-4-9b-chat"
model_dir = "../../train-result/2024-06-29/checkpoint-100"

query = "你好"

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()

Loading checkpoint shards: 100%|██████████| 8/8 [15:44<00:00, 118.02s/it]


In [3]:
gen_kwargs = {"max_length": 1024, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(result)

Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{"{"relevant APIs": [{"api_id": "0", "api_name": "查询代码", "required_parameters": ["api_0的结果"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "2", "api_name": "查询涨跌幅", "required_parameters": ["api_0的结果", "今日"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "3", "api_name": "查询涨跌幅", "required_parameters": ["api_0的结果", "今日"], "rely_apis": ["0"], "tool_name": "股票查询"}], "result": ["api_1的结果", "api_2的结果", "api_3的结果"]}api_4的结果"]}2", "3"], "rely_apis": ["4", "tool_name": "数值计算"}], "result": ["api_5的结果"]}api_3的结果"]}api_6的结果"]}api_4的结果"]}3", "6"], "rely_apis": ["3", "tool_name": "数值计算"}], "result": ["api_7的结果"]}api_5的结果"]}api_8的结果"]}7", "5"], "rely_apis": []}, {"api_id": "8", "tool_name": "数值计算"}], "result": ["api_9的结果"]}api_6的结果"]}api_7的结果"]}9"], "rely_apis": ["7", "8"], "tool_name": "数值计算"}], "result": ["api_8的结果"]}api_9的结果", "api_3的结果"]}api_8的结果"]}9"], "rely_apis": ["8"], "tool_name": "数值计算"}], "result": ["api_1的结果", "api_2的结果", "api_3的结果"]}api_9的结果"]}2", "3"], "rely_apis": []}, 

In [4]:
import pandas as pd
from tools.common_utils import highlight_diff, read_jsonl

json_file_path = 'data/dev.jsonl'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,messages
0,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
1,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
2,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
3,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
4,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."


In [5]:
from tools.standard_name_utils import optimize_parameters
data_stock = pd.read_excel('raw_data/标准名.xlsx',sheet_name='股票标准名')
data_fund = pd.read_excel('raw_data/标准名.xlsx',sheet_name='基金标准名')

fund_standard_name = data_fund['标准基金名称'].to_list()
stock_standard_name = data_stock['标准股票名称'].to_list()

In [8]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    single_user_row = row["messages"][0]
    single_assis_row = row["messages"][1]
    
    messages = [single_user_row]
    output = single_assis_row["content"]
    
    inputs = tokenizer.apply_chat_template([{"role": "user", "content": messages}], 
                                           add_generation_prompt=True, 
                                           tokenize=True, 
                                           return_tensors="pt", 
                                           return_dict=True)
    inputs = inputs.to(device)
    
    generate_kwargs = {"max_length": 1024, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    
    total_eval_count += 1
    if optimized_resp == output:
        correct_count += 1
        print("----corrected----")
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(messages)
        print("-----output diff-----")
        print(highlight_diff(output, optimized_resp))
        print(optimized_resp)
        print()
    
    if total_eval_count == 5:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
4
-----query input-----
[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：汇添富添添鑫A基金近1年的最大回撤及其同类排名信息是怎样的？ \n query中提到的产品标准名可能是：汇添富添添乐双鑫债券型证券投资基金C类、汇添富添添乐双鑫债券型证券投资基金A类、汇添富添添乐双盈债券型证券投资基金C类、汇添富添添乐双盈债券型证券投资基金A类、汇添富添富通货币市场基金E类、汇添富添富通货币市场基金E类、汇添富添富通货币市场基金B类、汇添富添富通货币市场基金A类、汇添富添添鑫多元收益9个月持有期混合型证券投资基金C类、汇添富添添鑫多元收益9个月持有期混合型证券投资基金A类、汇添富鑫瑞债券型证券投资基金C类、汇添富鑫瑞债券型证券投资基金A类、汇添富鑫汇债券型证券投资基金C类、汇添富鑫汇债券型证券投资基金A类、汇添富货币市场基金E类、汇添富货币市场基金D类、汇添富货币市场基金C类、汇添富货币市场基金B类、汇添富货币市场基金A类、汇添富鑫润纯债债券型证券投资基金C类、汇添富鑫润纯债债券型证券投资基金A类、汇添富双鑫添利债券型证券投资基金C类、汇添富双鑫添利债券型证券投资基金A类、汇添富盈鑫灵活配置混合型证券投资基金C类、汇添富盈鑫灵活配置混合型证券投资基金A类、汇添富保鑫灵活配置混合型证券投资基金C类、汇添富保鑫灵活配置混合型证券投资基金A类、汇添富长添利定期开放债券型证券投资基金C类、汇添富长添利定期开放债券型证券投资基金A类、汇添富鑫远债券型证券投资基金、汇添富鑫禧债券型证券投资基金、汇添富鑫福债券型证券投资基金、汇添富现金宝货币市场基金A类、汇添富和聚宝货币市场基金C类、汇添富和聚宝货币市场基金A类、汇添富全额宝货币市场基金A类、汇添富稳鑫90天持有期债券型证券投资基金C类、汇添富稳鑫90天持有期债券型证券投资基金A类、汇添富稳健添利定期开放债券型证券投资基金C类、汇添富稳健添利定期开放债券型证券投资基金A类、汇添富安鑫智选灵活配置混合型证券投资基金C类、汇添富安鑫智选灵活配置混合型证券投资基金A类、汇添富AAA级信用纯债债券型证券投资基金E类、汇添富AAA级信用纯债债券型证券投资基金C类、汇添富

Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
10
-----query input-----
[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我想用5000块钱买鹏华宁华A基金，若我现在申购，需要支付多少费用？ \n query中提到的产品标准名可能是：鹏华宁华一年持有期混合型证券投资基金C类、鹏华宁华一年持有期混合型证券投资基金A类、鹏华丰华债券型证券投资基金、鹏华增华混合型证券投资基金C类、鹏华增华混合型证券投资基金A类、人保中证500指数型证券投资基金、农银汇理中证500指数证券投资基金、华宝中证1000指数证券投资基金C类、华宝中证1000指数证券投资基金A类、鹏华弘华灵活配置混合型证券投资基金C类、鹏华弘华灵活配置混合型证券投资基金A类、华商中证500指数分级证券投资基金B类、华商中证500指数分级证券投资基金A类、信诚中证500指数分级证券投资基金B类、信诚中证500指数分级证券投资基金A类、中证500交易型开放式指数证券投资基金、鹏华鑫华一年持有期混合型证券投资基金C类、鹏华鑫华一年持有期混合型证券投资基金A类、鹏华浙华一年持有期混合型证券投资基金C类、鹏华浙华一年持有期混合型证券投资基金A类、鹏华招华一年持有期混合型证券投资基金C类、鹏华招华一年持有期混合型证券投资基金A类、鹏华中证500指数增强型证券投资基金C类、鹏华中证500指数增强型证券投资基金A类、长城中证500指数增强型证券投资基金C类、长城中证500指数增强型证券投资基金A类、长信中证500指数增强型证券投资基金C类、长信中证500指数增强型证券投资基金A类、财通中证500指数增强型证券投资基金C类、财通中证500指数增强型证券投资基金A类、诺安中证500指数增强型证券投资基金C类、诺安中证500指数增强型证券投资基金A类、浙商中证500指数增强型证券投资基金C类、浙商中证500指数增强型证券投资基金A类、汇安中证500指数增强型证券投资基金C类、汇安中证500指数增强型证券投资基金A类、招商中证500指数增强型证券投资基金C类、招商中证500指数增强型证券投资基金A类、建信中证500指数增强型证券投资基金C类、建信中证500指数增强型证券投资基金A类、广发中证500指数增

Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
52
-----query input-----
[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我想了解南方达元A最近一年的申购状态和风险等级 \n query中提到的产品标准名可能是：南方达元债券型证券投资基金A类、南方宝元债券型基金A类、南方贤元一年持有期债券型证券投资基金C类、南方贤元一年持有期债券型证券投资基金A类、南方永元一年持有期债券型证券投资基金C类、南方永元一年持有期债券型证券投资基金A类、鑫元一年定期开放中高等级债券型证券投资基金、南方景元中高等级信用债债券型证券投资基金C类、南方景元中高等级信用债债券型证券投资基金A类、南方达元债券型证券投资基金C类、南方纯元债券型证券投资基金A类、南方祥元债券型证券投资基金A类、南方启元债券型证券投资基金A类、南方双元债券型证券投资基金A类、南方卓元债券型证券投资基金A类、南方皓元短债债券型证券投资基金A类、南方梦元短债债券型证券投资基金A类、南方崇元纯债债券型证券投资基金A类、南方吉元短债债券型证券投资基金A类、南方浩达稳健优选一年持有期混合型基金中基金(FOF)C类、南方浩达稳健优选一年持有期混合型基金中基金(FOF)A类、南方高元债券型发起式证券投资基金A类、南方贺元利率债债券型证券投资基金A类、南方聪元债券型发起式证券投资基金A类、南方振元债券型发起式证券投资基金A类、南方定元中短债债券型证券投资基金A类、南方亨元债券型发起式证券投资基金A类、南方避险增值基金、易方达并购重组指数分级证券投资基金B类、易方达并购重组指数分级证券投资基金A类、南方恒庆一年定期开放债券型证券投资基金、南方定利一年定期开放债券型证券投资基金、南方利达灵活配置混合型证券投资基金A类、南方亚洲美元收益债券型证券投资基金A类、南方丰元信用增强债券型证券投资基金A类、南方富元稳健养老目标一年持有期混合型基金中基金(FOF)Y类、南方富元稳健养老目标一年持有期混合型基金中基金(FOF)C类、南方富元稳健养老目标一年持有期混合型基金中基金(FOF)A类、南方中债0-5年中高等级江苏省城投类债券指数证券投资基金C类、南方中债0-5年中高等级江苏省城投类债券指数证券投资基金A类、易方

Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
102
-----query input-----
[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：你知道有哪些基金的托管费率是0.19%吗？ \n query中提到的产品标准名可能是：博道睿见一年持有期混合型证券投资基金、博道盛兴一年持有期混合型证券投资基金、博道嘉兴一年持有期混合型证券投资基金、博道盛利6个月持有期混合型证券投资基金、博道安远6个月持有期混合型证券投资基金、申万菱信乐道三年持有期混合型证券投资基金、博道中证同业存单AAA指数7天持有期证券投资基金、博道和瑞多元稳健6个月持有期混合型证券投资基金C类、博道和瑞多元稳健6个月持有期混合型证券投资基金A类、鹏华沪深300交易型开放式指数证券投资基金(2013)、博道远航混合型证券投资基金C类、博道远航混合型证券投资基金A类、博道盛彦混合型证券投资基金C类、博道盛彦混合型证券投资基金A类、博道明远混合型证券投资基金C类、博道明远混合型证券投资基金A类、博道志远混合型证券投资基金C类、博道志远混合型证券投资基金A类、博道嘉瑞混合型证券投资基金C类、博道嘉瑞混合型证券投资基金A类、博道嘉泰回报混合型证券投资基金、博道嘉元混合型证券投资基金C类、博道嘉元混合型证券投资基金A类、博道嘉丰混合型证券投资基金C类、博道嘉丰混合型证券投资基金A类、博道启航混合型证券投资基金C类、博道启航混合型证券投资基金A类、博道卓远混合型证券投资基金C类、博道卓远混合型证券投资基金A类、博道久航混合型证券投资基金C类、博道久航混合型证券投资基金A类、鹏华国有企业债债券型证券投资基金、银华-道琼斯88精选证券投资基金、华夏瑞益混合型证券投资基金A1类、交银施罗德启道混合型证券投资基金、金洲管道、博道红利智航股票型证券投资基金C类、博道红利智航股票型证券投资基金A类、博道研究恒选混合型证券投资基金C类、博道研究恒选混合型证券投资基金A类、博道消费智航股票型证券投资基金C类、博道消费智航股票型证券投资基金A类、博道成长智航股票型证券投资基金C类、博道成长智航股票型证券投资基金A类、博道惠泰优选混合型证券投资基金C类、博道惠泰优选混合型证券投资基金A类、博道叁佰智航股票型证券投资基金

In [9]:
test_json_file_path = 'data/test_a.jsonl'
test_data_list = read_jsonl(test_json_file_path)

test_df = pd.DataFrame(test_data_list)
test_df.head()

,messages
0,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
1,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
2,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
3,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
4,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."


In [10]:
total_eval_count = 0

with open('data/submit.jsonl','w', encoding="utf-8") as n:
    for index, row in test_df.iterrows():  
        single_user_row = row["messages"][0]
        single_assis_row = row["messages"][1]
        
        messages = [single_user_row]
        output = single_assis_row["content"]
        
        inputs = tokenizer.apply_chat_template([{"role": "user", "content": messages}], 
                                           add_generation_prompt=True, 
                                           tokenize=True, 
                                           return_tensors="pt", 
                                           return_dict=True)
        inputs = inputs.to(device)
        
        generate_kwargs = {"max_length": 1024, "do_sample": True, "top_k": 1}
        with torch.no_grad():
            outputs = model.generate(**inputs, **gen_kwargs)
            outputs = outputs[:, inputs['input_ids'].shape[1]:]
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)        
        total_eval_count += 1
        
        n.write(optimized_resp+'\n')
        
        if total_eval_count % 10 == 0:
            print("现在是第" + f"{total_eval_count}" + "条数据")
            break

Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene

现在是第10条数据
